### Исследование файла и уравнений

In [1]:
with open("THERM.DAT", "r", encoding="UTF-8") as file:
        head = [next(file) for x in range(4)]
        for line in head:
            print(line)

Ag cr REF ELEMENT T 6/12AG 1.   0.   0.   0.S   200.000  1235.080 1000.        1

 2.07216824E+00 2.46393729E-03-1.34351116E-06 3.69321107E-10 0.00000000E+00    2

-6.37725170E+02-7.18810718E+00 2.25225065E+00 5.43263008E-03-1.32153990E-05    3

 1.50423505E-08-5.94991675E-12-8.23132027E+02-8.86835190E+00 0.00000000E+00    4



### Задача: распарсить файл по принципу:

`{"Ag" : [2.07216824E+00, 2.46393729E-03, -1.34351116E-06, 3.69321107E-10, 0.00000000E+00, -6.37725170E+02, -7.18810718E+00, 2.25225065E+00, 5.43263008E-03, -1.32153990E-05, 1.50423505E-08, -5.94991675E-12, -8.23132027E+02, -8.86835190E+00, 0.00000000E+00]}`

In [2]:
import json

In [3]:
def read_four_lines(f):
    first = f.readline()
    second = f.readline()
    third = f.readline()
    fourth = f.readline()
    return (first, second, third, fourth)

In [4]:
def parse_quadruple(q):
    element = q[0][0:q[0].index(' ')] #Надо очистить это от лишних символов (1,2-C2H2F2-cis)
    element = element.replace("*", "").replace("=", "")
    i1 = element.find("-")
    if i1 != -1:
        element = element[i1:]
    i2 = element.find("-")
    if i2 != -1:
        element = element[:i2]
    #C2H2F2
    meta = q[0][18:26]
    full_name = q[0][0:17].strip()
    coefs = []
    stride = 15
    for i in range(1, 4):
        for j in range(0, 5):
            raw_data = q[i][j*stride:(j+1)*stride]
            try:
                data = float(raw_data)
            except ValueError:
                data = float(0)
            coefs += [data]
    element_data = {element : {"coeffs" : coefs, "full_name" : full_name, "meta" : meta}}
    return element_data

In [5]:
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    q = read_four_lines(file)
    print(parse_quadruple(q))

{'Ag': {'coeffs': [2.07216824, 0.00246393729, -1.34351116e-06, 3.69321107e-10, 0.0, -637.72517, -7.18810718, 2.25225065, 0.00543263008, -1.3215399e-05, 1.50423505e-08, -5.94991675e-12, -823.132027, -8.8683519, 0.0], 'full_name': 'Ag cr REF ELEMENT', 'meta': 'T 6/12AG'}}


In [6]:
result = {}
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    for i in range(0, 2972): #Я вручную посчитал сколько там должно быть элементов
        q = read_four_lines(file)
        buf = parse_quadruple(q)
        result.update(buf)
parsed_to_json = json.dumps(result, indent=4)

In [7]:
print(result["H2"])

{'coeffs': [2.93286575, 0.000826608026, -1.46402364e-07, 1.54100414e-11, -6.888048e-16, -813.065581, -1.02432865, 2.34433112, 0.00798052075, -1.9478151e-05, 2.01572094e-08, -7.37611761e-12, -917.935173, 0.683010238, 0.0], 'full_name': 'H2 REF ELEMENT', 'meta': 'tpis78H '}


In [8]:
with open("parsed.json", "w", encoding="UTF-8") as file:
    file.write(parsed_to_json)

### Пример - H2 + N54.63O14.67

In [9]:
import sys
sys.path.insert(0,'../src/')
from astra import *

In [10]:
astra = Astra(components=["H2", "N54.64O14.67"])
#a = Astra(components=["O14.258H27.949N53.097"])

In [11]:
for component in astra.components:
    print(component)

[('H', 2.0)]
[('N', 54.64), ('O', 14.67)]


In [12]:
print(astra.product)

[('N', 54.64), ('H', 2.0), ('O', 14.67)]


In [13]:
print(astra.product_elements)

{'N', 'H', 'O'}


### Задача: выбрать из того файла все компоненты, у которых встречается в формуле любое из веществ выше, но нет других веществ

In [14]:
with open("parsed.json", "r", encoding="UTF-8") as file:
    text = file.read()
data = json.loads(text)

In [15]:
selected_elements = []
for key in data:
    table_elements = set(parse_to_element_list(key))
    my_elements = set(astra.product_elements)
    if table_elements.issubset(my_elements) and len(table_elements) != 0:
        print(table_elements)
        selected_elements += [key]

{'H'}
{'N', 'H', 'O'}
{'N', 'H', 'O'}
{'O', 'H', 'N'}
{'N', 'H', 'O'}
{'N', 'H', 'O'}
{'O', 'H'}
{'O', 'H'}
{'O', 'H'}
{'H'}
{'O', 'H'}
{'O', 'H'}
{'O', 'H'}
{'N'}
{'N', 'H'}
{'N', 'H', 'O'}
{'O', 'H', 'N'}
{'N', 'H'}
{'N', 'H', 'O'}
{'N', 'H', 'O'}
{'N', 'H'}
{'N', 'H', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N'}
{'N', 'H'}
{'N', 'H'}
{'N', 'H', 'O'}
{'N', 'H', 'O'}
{'N', 'H'}
{'N', 'H'}
{'N', 'H', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N', 'H', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N'}
{'N', 'H'}
{'N', 'H', 'O'}
{'N'}
{'O'}
{'O'}
{'O'}
{'O'}


In [16]:
print(selected_elements)

['H', 'HNO', 'HNO2', 'HONO', 'HNO3', 'HNO4', 'OH', 'HO2', 'HO3', 'H2', 'H2O', 'H2O2', 'HOOOH', 'N', 'NH', 'NOH', 'HOONO', 'NH2', 'NH2O', 'HNOH', 'NH3', 'NH2OH', 'NO', 'NO2', 'NOO', 'NO3', 'N2', 'N2H', 'N2H2', 'H2N2O', 'N2H2O2', 'N2H3', 'N2H4', 'NH4NO3', 'N2O', 'N2O2', 'HN2O2', 'N2O3', 'N2O4', 'N2O5', 'N3', 'N3H', 'HN3O4', 'N4', 'O', 'O2', 'O3', 'O4']


In [17]:
selected_elements_formulas = [parse_to_list(x) for x in selected_elements]
for selected_element in selected_elements_formulas:
    print(selected_element)

[('H', 1.0)]
[('H', 1.0), ('N', 1.0), ('O', 1.0)]
[('H', 1.0), ('N', 1.0), ('O', 2.0)]
[('H', 1.0), ('O', 2.0), ('N', 1.0)]
[('H', 1.0), ('N', 1.0), ('O', 3.0)]
[('H', 1.0), ('N', 1.0), ('O', 4.0)]
[('O', 1.0), ('H', 1.0)]
[('H', 1.0), ('O', 2.0)]
[('H', 1.0), ('O', 3.0)]
[('H', 2.0)]
[('H', 2.0), ('O', 1.0)]
[('H', 2.0), ('O', 2.0)]
[('H', 2.0), ('O', 3.0)]
[('N', 1.0)]
[('N', 1.0), ('H', 1.0)]
[('N', 1.0), ('O', 1.0), ('H', 1.0)]
[('H', 1.0), ('O', 3.0), ('N', 1.0)]
[('N', 1.0), ('H', 2.0)]
[('N', 1.0), ('H', 2.0), ('O', 1.0)]
[('H', 2.0), ('N', 1.0), ('O', 1.0)]
[('N', 1.0), ('H', 3.0)]
[('N', 1.0), ('H', 3.0), ('O', 1.0)]
[('N', 1.0), ('O', 1.0)]
[('N', 1.0), ('O', 2.0)]
[('N', 1.0), ('O', 2.0)]
[('N', 1.0), ('O', 3.0)]
[('N', 2.0)]
[('N', 2.0), ('H', 1.0)]
[('N', 2.0), ('H', 2.0)]
[('H', 2.0), ('N', 2.0), ('O', 1.0)]
[('N', 2.0), ('H', 2.0), ('O', 2.0)]
[('N', 2.0), ('H', 3.0)]
[('N', 2.0), ('H', 4.0)]
[('N', 2.0), ('H', 4.0), ('O', 3.0)]
[('N', 2.0), ('O', 1.0)]
[('N', 2.0), ('O'

### Задача: составить матрицу как в презентации

In [18]:
import numpy as np

In [19]:
b = np.array([x[1] for x in astra.product]).reshape(-1,1)

In [20]:
b # H O N

array([[54.64],
       [ 2.  ],
       [14.67]])

In [21]:
def find(l, value):
    try:
        index = l.index(value)
    except ValueError:
        index  = -1 
    return index


def make_line(component, product):
    row = []
    component_elements = [x[0] for x in component]
    for element, quantity in product:
        ind = find(component_elements, element)
        if ind!=-1:
            row += [component[ind][1]]
        else:
            row += [0]        
    return np.array(row)
            

In [22]:
print(make_line(selected_elements_formulas[2], astra.product))

[1. 1. 2.]


In [23]:
a = np.array([make_line(x, astra.product) for x in selected_elements_formulas]).T
#Не хватает еще уравнения состояния

In [24]:
print(a) 

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 4. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 0. 1. 1. 1. 2. 2. 2. 3. 3. 0. 0.
  0. 0. 0. 1. 2. 2. 2. 3. 4. 4. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 2. 2. 3. 4. 1. 2. 3. 0. 1. 2. 3. 0. 0. 1. 3. 0. 1. 1. 0. 1. 1. 2.
  2. 3. 0. 0. 0. 1. 2. 0. 0. 3. 1. 2. 2. 3. 4. 5. 0. 0. 4. 0. 1. 2. 3. 4.]]


### Задача решить систему `matrix b`

In [25]:
from numpy.linalg import lstsq

In [26]:
x = lstsq(a, b, rcond=-1)

In [27]:
x

(array([[-0.31920761],
        [ 0.28760684],
        [ 0.16035697],
        [ 0.16035697],
        [ 0.0331071 ],
        [-0.09414278],
        [-0.44645748],
        [-0.57370736],
        [-0.70095723],
        [-0.63841522],
        [-0.76566509],
        [-0.89291497],
        [-1.02016484],
        [ 0.73406433],
        [ 0.41485672],
        [ 0.28760684],
        [ 0.0331071 ],
        [ 0.09564911],
        [-0.03160077],
        [-0.03160077],
        [-0.2235585 ],
        [-0.35080838],
        [ 0.60681445],
        [ 0.47956458],
        [ 0.47956458],
        [ 0.35231471],
        [ 1.46812865],
        [ 1.14892104],
        [ 0.82971343],
        [ 0.70246356],
        [ 0.57521369],
        [ 0.51050582],
        [ 0.19129821],
        [-0.19045141],
        [ 1.34087878],
        [ 1.21362891],
        [ 0.8944213 ],
        [ 1.08637903],
        [ 0.95912916],
        [ 0.83187929],
        [ 2.20219298],
        [ 1.88298537],
        [ 1.37398588],
        [ 2

In [28]:
x[0]

array([[-0.31920761],
       [ 0.28760684],
       [ 0.16035697],
       [ 0.16035697],
       [ 0.0331071 ],
       [-0.09414278],
       [-0.44645748],
       [-0.57370736],
       [-0.70095723],
       [-0.63841522],
       [-0.76566509],
       [-0.89291497],
       [-1.02016484],
       [ 0.73406433],
       [ 0.41485672],
       [ 0.28760684],
       [ 0.0331071 ],
       [ 0.09564911],
       [-0.03160077],
       [-0.03160077],
       [-0.2235585 ],
       [-0.35080838],
       [ 0.60681445],
       [ 0.47956458],
       [ 0.47956458],
       [ 0.35231471],
       [ 1.46812865],
       [ 1.14892104],
       [ 0.82971343],
       [ 0.70246356],
       [ 0.57521369],
       [ 0.51050582],
       [ 0.19129821],
       [-0.19045141],
       [ 1.34087878],
       [ 1.21362891],
       [ 0.8944213 ],
       [ 1.08637903],
       [ 0.95912916],
       [ 0.83187929],
       [ 2.20219298],
       [ 1.88298537],
       [ 1.37398588],
       [ 2.93625731],
       [-0.12724987],
       [-0

In [29]:
print("Проверка: ", a @ x[0])

Проверка:  [[54.64]
 [ 2.  ]
 [14.67]]


In [30]:
elements_with_concentration = [(e, float(c)) for e, c in zip(selected_elements, x[0])]

In [31]:
for ewc in elements_with_concentration:
    print(ewc)

('H', -0.31920761023286115)
('HNO', 0.2876068436756128)
('HNO2', 0.16035697031274182)
('HONO', 0.16035697031274193)
('HNO3', 0.033107096949872225)
('HNO4', -0.09414277641299747)
('OH', -0.4464574835957331)
('HO2', -0.5737073569586029)
('HO3', -0.7009572303214725)
('H2', -0.6384152204657269)
('H2O', -0.7656650938285966)
('H2O2', -0.8929149671914662)
('HOOOH', -1.020164840554336)
('N', 0.7340643272713446)
('NH', 0.41485671703848126)
('NOH', 0.28760684367561157)
('HOONO', 0.033107096949872225)
('NH2', 0.09564910680561782)
('NH2O', -0.03160076655725187)
('HNOH', -0.03160076655725187)
('NH3', -0.22355850342724554)
('NH2OH', -0.3508083767901152)
('NO', 0.606814453908475)
('NO2', 0.4795645805456053)
('NOO', 0.4795645805456053)
('NO3', 0.3523147071827356)
('N2', 1.4681286545426893)
('N2H', 1.148921044309826)
('N2H2', 0.8297134340769625)
('H2N2O', 0.7024635607140929)
('N2H2O2', 0.5752136873512231)
('N2H3', 0.5105058238440991)
('N2H4', 0.19129821361123567)
('NH4NO3', -0.19045140647737335)
('N2O'

### Итоги

1. Файл обработан следующим образом:
     
     1.1 С первой строки получена формула элемента. Формула очищена от знаков `=` и `*`, а также удалено все до `-` и после него
     
     1.2 Для каждой следующей строки читается по 15 символов и записываются в список
     
     1.3 Пара формула:массив записывается в словарь
     
     1.4 Пункты 1.1-1.3 повторяются для каждых четверок строк в файле
     
     1.5 Полученный большой словарь переводится в json-строку и сохраняется в файл
     
     Проблемы: Не знаю, насколько правильно так обрабатывать формулу, Некоторые формулы повторяются и старые значения перезаписываются новыми
     
2. Решение системы уравнений происходит по следующим этапам:
    
    2.1 В конструктор класса Astra передается список формул
    
    2.2 Формулы соединяются, у одинаковых атомов складывается количество, расчета на стехиометрию пока нет
    
    2.3 Из файла выбираются те формулы, набор атомы которых являются подмножеством набора атомов соединенной формулы
    
    2.4 Для каждого атома (из набора атомов соединенной формулы) для каждой выбранной формулы находится содержание этого атома в этой формуле. Для каждой формулы получается список, который потом соединяется в матрицу размером (m, n), где m - количество формул, n - количество атомов в соединенной формуле
    
    2.5 Матрица транспонируется
    
    2.6 Аналогичное действие с нахождением количеств атомов делается с соединенной формулой
    
    2.7 Список количеств атомов в соединенной формуле транспонируется
    
    2.8 Уравнение `a @ x = b` решается решателем numpy.linalg.lstsq
    
    Проблемы: Не считается стехиометрическое соотношение, Не используются коэффициенты из файла, В решении есть отрицательные числа (Наверное они должны быть положительные, это же концентрации?), Нет замыкающего уравнения состояния (Пока не знаю как его добавить)